In [1]:
from scipy.sparse import csc_array
from scipy.sparse.linalg import bicgstab
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import gmres
import numpy as np
import matplotlib.pyplot as plt
pi = np.pi

In [2]:
def DatosMat(path):
    data = [[],[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

def DatosVec(path):
    data = [[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

In [3]:
Nx, Nt = 4, 4
block_x, block_t = 2, 2
x_elements, t_elements = int(Nx/block_x), int(Nt/block_t)
Ntest, Nagg = 1, 2*(block_x*block_t)
Ntot = Nx*Nt
Coords = np.zeros((Nx,Nt,2),dtype=int)
for x in range(Nx):
    for t in range(Nt):
        for s in range(2):
            Coords[x,t,s] = x * Nt * 2 + t * 2 + s
XCoord = np.zeros(2*Nx*Nt,dtype=int)
TCoord = np.zeros(2*Nx*Nt,dtype=int)
SCoord = np.zeros(2*Nx*Nt,dtype=int)
Agg = np.zeros((2*block_x*block_t,x_elements*t_elements),dtype=int)

In [44]:
Coords[0,0,1]

np.int64(1)

In [54]:
#Computing the aggregates
for x in range(block_x):
    for t in range(block_t):
        for s in range(2):
            x0, t0 = x * x_elements, t * t_elements
            x1, t1 = (x+1) * x_elements, (t+1) * t_elements
            aggregate = x * block_t * 2 + t * 2 + s
            count = 0
            for X in range(x0,x1):
                for T in range(t0,t1):
                    i = X * Nt * 2 + T * 2 + s
                    XCoord[i] , TCoord[i], SCoord[i] = X, T, s
                    Agg[aggregate][count] = i
                    count += 1

In [55]:
dim = 32
D = np.zeros((dim,dim),dtype=complex)
row, col, re, im = DatosMat("build/D.dat")
for i in range(dim*dim):
    D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 

In [239]:
class AMG():
    def __init__(self,D):
        self.test_vectors = np.zeros((Ntest,2*Ntot),dtype=complex) 
        self.P = np.zeros((2*Ntot,Ntest*Nagg),dtype=complex) 
        self.PH = np.zeros((Ntest*Nagg,2*Ntot),dtype=complex) 
        self.D = csc_array(D) #Dirac matrix
        self.Dc = np.zeros((Ntest*Nagg,Ntest*Nagg),dtype=complex) 

    def get_p(self):
        return self.P, self.PH

    def orthonormalize():
        #orthonormalize test vectors
        print('i have to finish this')

    def tv(self):  
        for i in range(Ntest): 
            for j in range(2*Ntot):
                theta = np.random.uniform(0,2*pi)
                self.test_vectors[i,j] = np.cos(theta) + 1j*np.sin(theta)
        for i in range(Ntest):
            self.test_vectors[i], exit_code = bicgstab(self.D, self.test_vectors[i], x0=self.test_vectors[i] ,maxiter=20,atol=1e-10)
            #print(self.test_vectors[i])
            print("gmres for test vectors",exit_code)
             
    def P_v(self):
        #Extracting the columns
        for can_vec in range(Ntest*Nagg):   
            P_v = np.zeros(2*Ntot,dtype=complex)
            v = np.eye(1,Ntest*Nagg,can_vec).reshape(Ntest*Nagg)
            for j in range(Ntest*Nagg):
                k = int(j / Nagg)
                a = int(j % Nagg)
                for i in range(len(Agg[a])):
                    x, t, s = XCoord[Agg[a,i]], TCoord[Agg[a,i]], SCoord[Agg[a,i]]
                    P_v[Coords[x,t,s]]  += self.test_vectors[k,Coords[x,t,s]] * v[j]
            self.P[:,can_vec] = P_v   
        self.PH = csc_array(np.conjugate(np.transpose(self.P)))
        self.P = csc_array(self.P)
        self.Dc = (PH @(D @ P)) #matrix multiplication

    def TwoGrid(self,nu1,nu2,max_iter,tol,x0,phi):
        x = x0
        err = 1
        k = 0
        r0 = phi - self.D.dot(x)
        err0 = print("intial error",np.real(np.vdot(r0,r0)))
        print(err)
        while(k < max_iter and err > tol):
            if nu1>0:
                x, exitCode = gmres(D, phi, x0=x ,maxiter=nu1,atol=1e-10)
           
            Pt_r = self.PH.dot( (phi-self.D.dot(x)))
            Dc_inv, exit_code = bicgstab(self.Dc, Pt_r,x0=Pt_r,maxiter=10000, atol=1e-10)#Dc^-1 P^H(phi-Dx)
            
            x += P.dot(Dc_inv)
            
            if nu2>0:
                x, exitCode = gmres(D, phi, x0=x ,maxiter=nu2,atol=1e-10)
            r = phi - self.D.dot(x)
            err = np.real(np.vdot(r,r)) #complex dot product
            print("iteration",k,"error",err)
            if err<tol:
                print("two-grid converged in",k+1,"iterations. Error ",err)
                return x
            k+=1
        print("two-grid did not converged in",max_iter,"iterations. Error ",err)
        
        return x

    def print_p(self):
        for i in range(2*Ntot):
            for j in range(Ntest*Nagg):
                print(self.P[i,j], end = " ")
            print('')

In [240]:
amg = AMG(D)
amg.tv()
amg.P_v()
P, PH = amg.get_p()
nu1, nu2 = 0, 1
max_iter = 5
tol = 1e-10
x0 = np.zeros(2*Ntot,dtype=complex)
for i in range(2*Ntot):  
    theta = np.random.uniform(0,2*pi)
    x0[i] = np.cos(theta) + 1j*np.sin(theta)
phi = x0
amg.TwoGrid(nu1,nu2,max_iter,tol,x0,phi)

gmres for test vectors 20
intial error 64.26429447052638
1
iteration 0 error 51.35852601815458
iteration 1 error 38.39892298750655
iteration 2 error 30.46136564305855
iteration 3 error 17.75067984670595
iteration 4 error 10.470338403392407
two-grid did not converged in 5 iterations. Error  10.470338403392407


array([ 2.51068137e-02 +0.74508984j,  3.34825356e+00 +0.22181899j,
       -8.03673440e+00 -3.23495712j, -1.35589331e+01 -1.94734812j,
        2.52502127e+01 -0.5788894j ,  1.89556155e+01-19.04133673j,
        9.84220529e+00 +2.87714337j,  1.51874872e+01-14.26037767j,
       -1.22605870e+01 +3.15903671j,  6.48273899e-01 +1.51688375j,
       -2.12635824e+00 -8.94793777j, -2.30739063e+00 +2.85941597j,
       -1.51162614e+00-17.13295822j,  5.44546814e+00-23.98838392j,
        1.71071488e+01+17.10426059j, -8.45788491e+00 +2.6361799j ,
        2.86033554e+00-13.51327928j, -1.36581879e+01-18.33144776j,
        3.76121639e+00 -0.15448996j,  5.01973218e+00 -8.80314484j,
       -1.21428535e+01+51.2131546j , -1.65696889e+01+17.90081887j,
        2.61109413e+01+49.21670114j, -1.76384589e+01 +3.97172025j,
        3.43098533e-01 +5.7392164j ,  3.06009510e+01+17.52031689j,
        4.81204762e+00 +3.41501708j,  2.34279914e+01-23.07917321j,
        2.79749370e+01+47.46137387j,  3.04638781e+01 -3.747157

In [241]:
Dsparse = csc_array(D)
x = x0
for i in range(1000):
    x, exit_code = gmres(Dsparse, phi, x0=x, maxiter=1,atol=1e-10)
    r = phi - Dsparse.dot(x)
    err = np.real(np.vdot(r,r))
    print(i,err,exit_code)

0 51.35852601815401 1
1 2.1520887973109457 1
2 0.24677546261944924 1
3 0.011790551711513796 1
4 0.0004108945355000157 1
5 3.891928203127668e-05 1
6 7.789637530303032e-06 1
7 2.1493821223599975e-07 1
8 1.1107835286105327e-07 0
9 1.1107835286105327e-07 0
10 1.1107835286105327e-07 0
11 1.1107835286105327e-07 0
12 1.1107835286105327e-07 0
13 1.1107835286105327e-07 0
14 1.1107835286105327e-07 0
15 1.1107835286105327e-07 0
16 1.1107835286105327e-07 0
17 1.1107835286105327e-07 0
18 1.1107835286105327e-07 0
19 1.1107835286105327e-07 0
20 1.1107835286105327e-07 0
21 1.1107835286105327e-07 0
22 1.1107835286105327e-07 0
23 1.1107835286105327e-07 0
24 1.1107835286105327e-07 0
25 1.1107835286105327e-07 0
26 1.1107835286105327e-07 0
27 1.1107835286105327e-07 0
28 1.1107835286105327e-07 0
29 1.1107835286105327e-07 0
30 1.1107835286105327e-07 0
31 1.1107835286105327e-07 0
32 1.1107835286105327e-07 0
33 1.1107835286105327e-07 0
34 1.1107835286105327e-07 0
35 1.1107835286105327e-07 0
36 1.11078352861053

In [185]:
PH.dot(np.random.rand(32)) + 4

array([3.47465901-0.31944128j, 4.10532816-3.05228299j,
       1.03587163-1.5396962j , 6.38740084+2.80944439j,
       3.30092143-0.60738601j, 5.34088931-6.38788203j,
       3.91962568-1.98264262j, 4.72666298-5.07108991j])

In [183]:
PH.shape

(8, 32)